<a href="https://colab.research.google.com/github/Songnahyun/Capstone-Design/blob/main/audiofinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Usual Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import os

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zero_crossings_mean=[]
zero_crossings_var = []
y_perc_mean=[] 
y_perc_var=[]
y_harm_mean=[] 
y_harm_var=[]
spectral_centroids_mean=[] 
spectral_centroids_var=[]
spectral_rolloff_mean=[]
spectral_rolloff_var=[]
chromagram_mean=[]
chromagram_var=[]
mfcc_mean=[]
mfcc_var=[]
labels= []

In [ ]:
def AudioFeatures(music):
    wav, sr = librosa.load(f'/content/drive/MyDrive/baby/웃음/{music}')

    # Zero-crossing
    zero_crossings = librosa.zero_crossings(wav, pad=False)
    zero_crossings_mean.append(zero_crossings.mean())
    zero_crossings_var.append(zero_crossings.var())

    #Harmonics and Perceptrual
    y_harm, y_perc = librosa.effects.hpss(wav)
    y_harm_mean.append(y_harm.mean())
    y_harm_var.append(y_harm.var())
    y_perc_mean.append(y_perc.mean())
    y_perc_var.append(y_perc.var())


    #Spectral Centroid / Rolloff
    spectral_centroids = librosa.feature.spectral_centroid(wav, sr=sr)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(wav, sr=sr)[0]

    spectral_centroids_mean.append(spectral_centroids.mean())
    spectral_centroids_var.append(spectral_centroids.var())
    spectral_rolloff_mean.append(spectral_rolloff.mean())
    spectral_rolloff_var.append(spectral_rolloff.var())

    #Mel-Frequency Cepstral Coefficients
    mfccs = librosa.feature.mfcc(wav, sr=sr)
    mfccs = sklearn.preprocessing.scale(mfccs, axis=1)
    mfcc_mean.append(mfccs.mean())
    mfcc_var.append(mfccs.var())

    #Chroma Frequencies
    hop_length = 5000
    chromagram = librosa.feature.chroma_stft(wav, sr=sr, hop_length=hop_length)
    chromagram_mean.append(chromagram.mean())
    chromagram_var.append(chromagram.var())

    #Label
    labels.append('웃음')

In [ ]:
musics = list(os.listdir('/content/drive/MyDrive/baby/웃음'))
for i in musics:
    AudioFeatures(i)

In [ ]:
music_df = {"zero_crossings_mean": zero_crossings_mean,"zero_crossings_var": zero_crossings_var ,"y_perc_mean": y_perc_mean, 
         "y_perc_var": y_perc_var, "y_harm_mean":y_harm_mean,"y_harm_var" : y_harm_var,"spectral_centroids_mean" : spectral_centroids_mean,
         "spectral_centroids_var" : spectral_centroids_var,"spectral_rolloff_mean" :spectral_rolloff_mean,"spectral_rolloff_var" : spectral_rolloff_var,
         "chromagram_mean" :chromagram_mean, "chromagram_var" :chromagram_var ,"mfcc_mean" :mfcc_mean, "mfcc_var" : mfcc_var, "labels" : labels 
        }
df = pd.DataFrame(music_df)

In [ ]:
df

,zero_crossings_mean,zero_crossings_var,y_perc_mean,y_perc_var,y_harm_mean,y_harm_var,spectral_centroids_mean,spectral_centroids_var,spectral_rolloff_mean,spectral_rolloff_var,chromagram_mean,chromagram_var,mfcc_mean,mfcc_var,labels
0,0.134099,0.116117,-1.564613e-05,0.002869,-1.835442e-04,0.005026,2612.334007,7.344494e+05,5223.890154,4.116898e+06,0.340510,0.113237,-2.492013e-09,1.0,배고픔
1,0.188170,0.152762,-1.188729e-07,0.000174,-1.491888e-05,0.000180,3231.706157,8.240858e+05,6502.501288,3.256921e+06,0.332817,0.107560,-8.283816e-10,1.0,배고픔
2,0.161389,0.135343,2.965975e-06,0.003134,3.226794e-07,0.002150,2423.508555,2.991891e+05,4100.770153,2.725039e+06,0.357014,0.102528,3.251162e-09,1.0,배고픔
3,0.146283,0.124884,-2.047325e-06,0.001807,-5.609354e-08,0.001487,2299.881586,2.358952e+05,3893.837919,1.455174e+06,0.337139,0.107211,-4.134426e-10,1.0,배고픔
4,0.099271,0.089416,-8.880363e-05,0.000396,-2.212612e-04,0.001270,1859.064359,5.046534e+05,3380.038499,3.162618e+06,0.265662,0.099599,1.467191e-09,1.0,배고픔
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.214333,0.168394,2.092527e-06,0.008663,3.098265e-07,0.008639,3572.258184,6.953015e+05,6923.443677,8.140047e+05,0.319706,0.096585,-1.723508e-09,1.0,웃음
184,0.154729,0.130788,2.658205e-05,0.000961,-3.297495e-07,0.000442,2970.014356,1.956704e+06,5109.777145,4.946576e+06,0.459420,0.114288,-5.113535e-09,1.0,웃음
185,0.228391,0.176229,1.608571e-06,0.001408,-1.231361e-06,0.000045,3973.284269,1.277734e+06,6919.478272,2.550976e+06,0.549129,0.113220,0.000000e+00,1.0,웃음
186,0.151982,0.128884,4.180886e-05,0.002259,2.511648e-07,0.000492,2547.620233,1.046435e+06,4598.009611,2.694087e+06,0.391782,0.109242,-4.097419e-09,1.0,웃음


In [ ]:
df['labels'].unique()

array(['배고픔', '아픔', '졸림', '웃음'], dtype=object)

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

In [ ]:
y = df['labels']
X = df.loc[:, df.columns != 'labels']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X_train)

# new data frame with the new scaled data. 
XScaled = pd.DataFrame(np_scaled, columns = cols)

In [ ]:
cols

Index(['zero_crossings_mean', 'zero_crossings_var', 'y_perc_mean',
       'y_perc_var', 'y_harm_mean', 'y_harm_var', 'spectral_centroids_mean',
       'spectral_centroids_var', 'spectral_rolloff_mean',
       'spectral_rolloff_var', 'chromagram_mean', 'chromagram_var',
       'mfcc_mean', 'mfcc_var'],
      dtype='object')

In [ ]:
TestScaled = min_max_scaler.transform(X_test)
TestScaled = pd.DataFrame(TestScaled, columns = cols)

In [ ]:
def model_assess(model,title = "Default"):
    model.fit(XScaled, y_train)
    preds = model.predict(TestScaled)
    #print(confusion_matrix(y_test, preds))
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

In [ ]:
nb = GaussianNB()
model_assess(nb, "Naive Bayes")

# Stochastic Gradient Descent
sgd = SGDClassifier(max_iter=5000, random_state=0)
model_assess(sgd, "Stochastic Gradient Descent")

# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, "KNN")

# Decission trees
tree = DecisionTreeClassifier()
model_assess(tree, "Decission trees")

# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, "Random Forest")

# Support Vector Machine
svm = SVC(decision_function_shape="ovo")
model_assess(svm, "Support Vector Machine")

# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model_assess(lg, "Logistic Regression")

# Neural Nets
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5000, 10), random_state=1)
model_assess(nn, "Neural Nets")

# Cross Gradient Booster
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_assess(xgb, "Cross Gradient Booster")

# Cross Gradient Booster (Random Forest)
xgbrf = XGBRFClassifier(objective= 'multi:softmax')
model_assess(xgbrf, "Cross Gradient Booster (Random Forest)")

Accuracy Naive Bayes : 0.80851 

Accuracy Stochastic Gradient Descent : 0.7234 

Accuracy KNN : 0.74468 

Accuracy Decission trees : 0.78723 

Accuracy Random Forest : 0.85106 

Accuracy Support Vector Machine : 0.85106 

Accuracy Logistic Regression : 0.78723 

Accuracy Neural Nets : 0.78723 

Accuracy Cross Gradient Booster : 0.85106 

Accuracy Cross Gradient Booster (Random Forest) : 0.80851 



In [ ]:
pd.crosstab(y_test,rforest.predict(TestScaled))

col_0,배고픔,아픔,웃음,졸림
labels,,,,
배고픔,9,0,1,0
아픔,0,8,2,0
웃음,1,0,11,0
졸림,0,0,3,12


In [ ]:
best_score = 0.85106

#임의의 gamma와 C를 지정해 반복문으로 돌려봄

for gamma in [0.001,0.01,0.1,1,10,100]:
  for C in [0.001,0.01,0.1,1,10,100]:
    svm = SVC(gamma = gamma, C = C)
    svm.fit(X_train, y_train)

    test_score = svm.score(X_test, y_test)

    if test_score > best_score:
      best_score = test_score
      best_prameter = {'gamma': gamma, 'C': C}
      train_score = svm.score(X_train, y_train)

print("Best score : {}".format(np.round(best_score,3)))
print("Parameters : {}".format(best_prameter))

Best score : 0.872
Parameters : {'gamma': 0.001, 'C': 1}


In [ ]:
param_grid = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [None, 6, 9, 12],
    'min_samples_split': [0.01, 0.05, 0.1],
    'max_features': ['auto', 'sqrt'],
}

estimator = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(random_state=30,
           n_splits=10,
           shuffle=True,
          )

In [ ]:
from sklearn.model_selection import GridSearchCV
# define grid_search
grid_search = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=kf, 
                           n_jobs=-1, 
                           verbose=2
                          )

# fit with (x_train, y_train)
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


GridSearchCV(cv=KFold(n_splits=10, random_state=30, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 6, 9, 12],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_split': [0.01, 0.05, 0.1],
                         'n_estimators': [100, 150, 200, 250]},
             verbose=2)

In [ ]:
grid_search.best_params_

{'max_depth': 6,
 'max_features': 'sqrt',
 'min_samples_split': 0.05,
 'n_estimators': 100}

In [ ]:
rforest = RandomForestClassifier (max_depth= 6, 
                                  max_features= 'sqrt', 
                                  min_samples_split= 0.05, 
                                  n_estimators= 100)
model_assess(rforest, "Random Forest")

Accuracy Random Forest : 0.85106 

